In [119]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
import pandas as pd  #import the necessary libraries.
import numpy as np 
import math as mt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.2f}".format
from datetime import datetime

In [121]:
df= pd.read_excel("/content/drive/MyDrive/BRISA-SampleData.xlsx", sheet_name=0)

In [122]:
dft=df.copy(deep=True)

In [123]:
df = df.fillna(0)

In [124]:
df.loc[:,'JANUARY':'DECEMBER']
df.columns = df.columns.str.replace('OCK', '01')
df.columns = df.loc[4:14].columns.str.replace('SBT', '02')
df.columns = df.loc[4:14].columns.str.replace('MAR', '03')
df.columns = df.columns.str.replace('NIS', '04')
df.columns = df.columns.str.replace('MAY', '05')
df.columns = df.columns.str.replace('HAZ', '06')
df.columns = df.columns.str.replace('TEM', '07')
df.columns = df.columns.str.replace('AGU', '08')
df.columns = df.columns.str.replace('EYL', '09')
df.columns = df.columns.str.replace('EKM', '10')
df.columns = df.columns.str.replace('KSM', '11')
df.columns = df.columns.str.replace('ARL', '12')
df.columns = df.columns.str.replace(' ','-')

df.rename({'03CH' : "MARCH",
            '05' : "MAY",
             'SEP07BER' : "SEPTEMBER",},axis=1, inplace=True)


In [125]:
df.columns

Index(['speccode', 'main_group', 'stok', 'RESGROUPCODE', '08-2022', '12-2022',
       '10-2022', '09-2022', '06-2022', '11-2022', '05-2022', '07-2022',
       'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY',
       'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER'],
      dtype='object')

In [126]:
if(df.iloc[: , -1].name=='DECEMBER.1'):
  first_cols = ['speccode', 'main_group', 'stok', 'RESGROUPCODE','JANUARY',
 'FEBRUARY',
 'MARCH',
 'APRIL',
 'MAY',
 'JUNE',
 'JULY',
 'AUGUST',
 'SEPTEMBER',
 'OCTOBER',
 'NOVEMBER',
 'DECEMBER',
 'DECEMBER.1']
  sec_cols= ['JANUARY', 
  'FEBRUARY',
  'MARCH',
  'APRIL',
  'MAY',
  'JUNE',
  'JULY',
  'AUGUST',
  'SEPTEMBER',
  'OCTOBER',
  'NOVEMBER',
  'DECEMBER',
  'DECEMBER.1']
  other_cols_2= df.columns[0:4].difference(sec_cols).tolist()
  other_cols = np.sort(df.columns.difference(first_cols)).tolist()
  other_cols= sorted(other_cols, key=lambda x: datetime.strptime(x, '%m-%Y'))
  df = df.loc[:, other_cols_2+ other_cols+ sec_cols]
  
else:
  first_cols = ['speccode', 'main_group', 'stok', 'RESGROUPCODE','JANUARY',
 'FEBRUARY',
 'MARCH',
 'APRIL',
 'MAY',
 'JUNE',
 'JULY',
 'AUGUST',
 'SEPTEMBER',
 'OCTOBER',
 'NOVEMBER',
 'DECEMBER']
  sec_cols= ['JANUARY', 
  'FEBRUARY',
  'MARCH',
  'APRIL',
  'MAY',
  'JUNE',
  'JULY',
  'AUGUST',
  'SEPTEMBER',
  'OCTOBER',
  'NOVEMBER',
  'DECEMBER']
  other_cols_2= df.columns[0:4].difference(sec_cols).tolist()
  other_cols = np.sort(df.columns.difference(first_cols)).tolist()
  other_cols= sorted(other_cols, key=lambda x: datetime.strptime(x, '%m-%Y'))

  df = df.loc[:, other_cols_2+ other_cols+ sec_cols]


In [127]:
filter2 = df.groupby(["RESGROUPCODE"])[other_cols].sum()
df = pd.merge(df, filter2, on ='RESGROUPCODE', how ='inner')

In [128]:
k = df.columns.get_loc("JANUARY")
wc_list=df.iloc[:,3:k].to_numpy() 

In [129]:
temp=[]
list1=[]
wc=[]
for i in range(len(wc_list)):
    temp=list(wc_list[i])
    for j in range(0,2*len(temp)-2):
        if (j%2==0):
            temp.insert(j+2,temp[j]-temp[j+1])
    list1.append(temp)
    for k in range(0,len(temp)):
        if ((list1[i][k]<0)):
                wc.append((k-2)/2+(list1[i][k-2]/list1[i][k-1]))
                break
        elif (list1[i][0]==list1[i][len(temp)-1]): #not forecasted
                wc.append(20)
                break
        elif (( list1[i][0]>abs(list1[i][len(temp)-1]) and (list1[i][len(temp)-1]>0) )): #having extra inventory
                wc.append(14)
                break

In [130]:
df=df.assign(WC=wc)
df['WC']=df['WC']*4
df[df['WC']==56]
filter1= df[df['WC']<=56]
filter1.reset_index(inplace=True,drop=True)

In [131]:
if(df.iloc[: , -1].name=='DECEMBER.1'):

  m = df.columns.get_loc("DECEMBER.1") + 1
else:
  m = df.columns.get_loc("DECEMBER") + 1
l = df.columns.get_loc("WC")
n = l - m
s = m - n + 1
capacity_utilization_oe=(filter1.iloc[:,m+1:l].values / filter1.iloc[:,s:m].values) #(forecast/production_capacity)

In [132]:
cudf=pd.DataFrame(capacity_utilization_oe).add_suffix('_capacity_utilization')

In [133]:
filter1= filter1.join((cudf))

In [134]:
p = l + 1
t=filter1.iloc[:,p:].values
t=t.tolist()

In [135]:
liste=[]
temp=(len(t[0]))/2
if (len(t[0])%2!=0):
    for i in range(len(t)):
        s=np.append(t[i],0)
        liste.append(s)
    for j in range(len(t)):
        liste[j]=np.array(liste[j]).reshape(((len(t[0])//2)+1),2)
else:
    for i in range(len(t)):
        s=t[i]
        liste.append(s)
    for j in range(len(t)):
        liste[j]=np.array(liste[j]).reshape((len(t[0])//2),2)

In [136]:
weight_1 = 0.6
weight_2 = 0.4

In [137]:
CU_cal=[]
j=0
for i in range (len(t)):
    while(len(t[i])!=0):
        if (len(t[i])>=4):
            max_two=max(t[i][j],t[i][j+1])
            max_two_2=max(t[i][j+2],t[i][j+3])
            weighted=weight_1*max_two+weight_2*max_two_2
            CU_cal.append(weighted)
            t[i].pop(0)
        if (len(t[i])==3):
            weighted=max(t[i][j],t[i][j+1])*weight_1+weight_2*t[i][j+2]
            CU_cal.append(weighted)
            t[i].pop(0)
        if (len(t[i])==2):
            weighted=t[i][j]*weight_1+t[i][j+1]*weight_2
            CU_cal.append(weighted)
            t[i].pop(0)
        if (len(t[i])==1):
            weighted=t[i][j]
            CU_cal.append(weighted)
            t[i].pop(0)

In [138]:
twcr1=np.array(CU_cal).reshape(len(liste),n-1)

In [139]:
twc1 = 2
twc2 = 3
twc3 = 4
twc4 = 5

WC_2= 0.25
WC_3= 0.5
WC_4= 0.75
WC_5= 1

In [140]:
def TWCR(twcr1,WC_2,WC_3,WC_4,WC_5,twc1,twc2,twc3,twc4):
    WC_2=float(WC_2)
    WC_3=float(WC_3)
    WC_4=float(WC_4)
    WC_5=float(WC_5)
    twc=[]
    for i in range(len(twcr1)):
        for j in range(len(twcr1[0])): 
            
            if (twcr1[i][j]>=0 and twcr1[i][j]<= WC_2):
                twc.extend([twc1])
    
            elif (twcr1[i][j]>WC_2 and twcr1[i][j] <= WC_3):
                twc.extend([twc2])
    
            elif (twcr1[i][j]>WC_3 and twcr1[i][j] <= WC_4):
                twc.extend([twc3])
    
            else:
                twc.extend([twc4])
    
    twc=np.array(twc).reshape(len(liste),n-1)
    return twc

In [141]:
TWCR=TWCR(twcr1,WC_2,WC_3,WC_4,WC_5,twc1,twc2,twc3,twc4) #targeted wc values are changing from month to month

In [142]:
ndf=filter1.copy(deep=True)
omdf = pd.DataFrame(TWCR).add_suffix('_TWC')

ndf = ndf.join(omdf)

In [143]:
#prod_gr = "PSR"

In [144]:
filter3 = ndf
#ndf[ndf['main_group']==prod_gr] #Change the production group. 


In [145]:
if(df.iloc[: , -1].name=='DECEMBER.1'):

  m = df.columns.get_loc("DECEMBER.1") + 1
else:
  m = df.columns.get_loc("DECEMBER") + 1

l = filter3.columns.get_loc("WC")
k = l + 1
s = filter3.columns.get_loc("0_TWC")
n = l - m - 1

x=filter3.iloc[:,k:s].values.tolist()
wc=filter3['WC'].values.tolist()
prod_group=filter3['main_group'].tolist()
prod_code=filter3['speccode'].tolist()

WC_targeted_1month=filter3.iloc[:,-n:].values.tolist()
forecasted=filter3.iloc[:,m+1:l].values.tolist()

In [146]:
if(dft.iloc[: , -1].name!='DECEMBER.1'):
    other_cols.pop(0)


In [147]:
ndf = ndf[ndf.columns.drop(list(ndf.filter(regex='_x')))]

In [148]:
ndf.columns

Index(['RESGROUPCODE', 'main_group', 'speccode', 'stok', 'JANUARY', 'FEBRUARY',
       'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER',
       'OCTOBER', 'NOVEMBER', 'DECEMBER', '05-2022_y', '06-2022_y',
       '07-2022_y', '08-2022_y', '09-2022_y', '10-2022_y', '11-2022_y',
       '12-2022_y', 'WC', '0_capacity_utilization', '1_capacity_utilization',
       '2_capacity_utilization', '3_capacity_utilization',
       '4_capacity_utilization', '5_capacity_utilization',
       '6_capacity_utilization', '0_TWC', '1_TWC', '2_TWC', '3_TWC', '4_TWC',
       '5_TWC', '6_TWC'],
      dtype='object')

In [149]:
#uncomment if you want to download the end data.
ndf.to_excel('ndf.xlsx') #downloading the data
from google.colab import files
files.download('ndf.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>